

## SENTIMENT ANALYSIS & PART OF SPEECH TAGGING

In [2]:
# importing dependencies here
import numpy as np
import pandas as pd
import os

# feature engineering
import re

# pos tagging
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# sentiment scoring
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# scaling to handle negative values in sentiment scores (for Naive Bayes)
from sklearn.preprocessing import MinMaxScaler

# performance check
import time

# code formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

### Feature Engineering - II

In [3]:
# reading the clean_dataset_1
personality_data = pd.read_csv(os.path.join("..", "data", "clean_data_1.csv"))

<IPython.core.display.Javascript object>

In [5]:
# looking at the top 5 rows of the dataset
personality_data.head()

,type,is_Extrovert,is_Sensing,is_Thinking,is_Judging,posts,clean_posts
0,INFJ,0,0,0,1,'When asked of the things you wish you did ear...,when asked the things you wish you did earli...
1,INFJ,0,0,0,1,'I love both and they are equally important to...,love both and they are equally important ...
2,INFJ,0,0,0,1,Really? You think implying that everyone who i...,really you think implying that everyone who i...
3,ENFJ,1,0,0,1,'Love is a crazy thing. Se is our best form ...,love crazy thing our best form commun...
4,INTP,0,0,1,0,'I am a physics undergrad with a computation e...,physics undergrad with computation emphas...


In [6]:
# checking the number of rows and columns
personality_data.shape

(8588, 7)

#### Checking for Null values

In [7]:
# checking for missing values
personality_data.isnull().sum()

type            0
is_Extrovert    0
is_Sensing      0
is_Thinking     0
is_Judging      0
posts           0
clean_posts     0
dtype: int64

There are no missing values present in this dataset.

### Sentiments Analysis Score

* CAUTION - Sentiment scoring will take LONG !!

In [9]:
# sentiment scoring for each user
t = time.time()

analyzer = SentimentIntensityAnalyzer()

nlp_sentiment_score = []

for post in personality_data["clean_posts"]:
    score = analyzer.polarity_scores(post)
    nlp_sentiment_score.append(score)

print(f"Sentiment Scoring Time: {time.time() - t:.2f} seconds")

Sentiment Scoring Time: 604.15 seconds


In [10]:
# segregating the indiviual sentiment scores - compound, positive, negative and neutral
personality_data["compound_sentiment"] = [
    score["compound"] for score in nlp_sentiment_score
]
personality_data["pos_sentiment"] = [score["pos"] for score in nlp_sentiment_score]
personality_data["neg_sentiment"] = [score["neg"] for score in nlp_sentiment_score]
personality_data["neu_sentiment"] = [score["neu"] for score in nlp_sentiment_score]

In [11]:
# Sentiment scores have negative values that Naive Bayes can't handle. So scaling it.

min_max_scaler = MinMaxScaler()
personality_data["compound_sentiment"] = min_max_scaler.fit_transform(
    np.array(personality_data["compound_sentiment"]).reshape(-1, 1)
)
personality_data["pos_sentiment"] = min_max_scaler.fit_transform(
    np.array(personality_data["pos_sentiment"]).reshape(-1, 1)
)
personality_data["neg_sentiment"] = min_max_scaler.fit_transform(
    np.array(personality_data["neg_sentiment"]).reshape(-1, 1)
)
personality_data["neu_sentiment"] = min_max_scaler.fit_transform(
    np.array(personality_data["neu_sentiment"]).reshape(-1, 1)
)

In [12]:
# checking to see if sentiment scores introduced any null value
personality_data.isnull().sum()

type                  0
is_Extrovert          0
is_Sensing            0
is_Thinking           0
is_Judging            0
posts                 0
clean_posts           0
compound_sentiment    0
pos_sentiment         0
neg_sentiment         0
neu_sentiment         0
dtype: int64

### POS Tagging

In [13]:
# creating tag_posts column that will have each post as a separate list in a row. tag_posts will be a list of 50 lists.

# replacing urls with domain name
personality_data["tag_posts"] = personality_data["posts"].str.replace(
    re.compile(r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*"),
    lambda match: match.group(2),
)

# replacing ||| with space
personality_data["tag_posts"] = [
    post for post in personality_data["tag_posts"].str.split("\|\|\|")
]

* CAUTION - The next step i.e. Parts of speech tagging for each word will take SUPER LONG !!!

In [23]:
# parts of speech tagging for each word
t = time.time()

personality_data["tagged_words"] = personality_data["tag_posts"].apply(
    lambda x: [nltk.pos_tag(word_tokenize(line)) for line in x]
)

print(f"POS Tagging Time: {time.time() - t} seconds")

POS Tagging Time: 2230.3588178157806 seconds


In [24]:
# creating list of unique POS tags
tag_set = set()

for i, data in personality_data["tagged_words"].iteritems():
    for tup in data[0]:
        tag_set.add(tup[1])

tag_list = list(tag_set)

C:\Users\Huawei D15\AppData\Local\Temp\ipykernel_18108\2273190258.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, data in personality_data["tagged_words"].iteritems():


In [25]:
# calculating mean and standard deviation of pos tags for each user
t = time.time()


def pos_cat(x, tag):
    return [len([y for y in line if y[1] == tag]) for line in x]


for col in tag_list:
    personality_data["POS_" + col + "_mean"] = personality_data["tagged_words"].apply(
        lambda x: np.mean(pos_cat(x, col))
    )
    personality_data["POS_" + col + "_std"] = personality_data["tagged_words"].apply(
        lambda x: np.std(pos_cat(x, col))
    )

print(f"POS Stats Time: {time.time() - t} seconds")

POS Stats Time: 321.5249733924866 seconds


In [26]:
# grouping pos tags based on stanford list
tags_dict = {
    "ADJ": ["JJ", "JJR", "JJS"],
    "ADP": ["EX", "TO"],
    "ADV": ["RB", "RBR", "RBS", "WRB"],
    "CONJ": ["CC", "IN"],
    "DET": ["DT", "PDT", "WDT"],
    "NOUN": ["NN", "NNS", "NNP", "NNPS"],
    "NUM": ["CD"],
    "PRT": ["RP"],
    "PRON": ["PRP", "PRP$", "WP", "WP$"],
    "VERB": ["MD", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
    ".": ["#", "$", "''", "(", ")", ",", ".", ":"],
    "X": ["FW", "LS", "UH"],
}

In [27]:
# Stanford POS tag stats
t = time.time()


def stanford_tag(x, tag):
    tags_list = [len([y for y in line if y[1] in tags_dict[col]]) for line in x]
    return tags_list


for col in tags_dict.keys():
    personality_data[col + "_avg"] = personality_data["tagged_words"].apply(
        lambda x: np.median(stanford_tag(x, col))
    )

print(f"Stanford POS Stats Time: {time.time() - t} seconds")

C:\Users\Huawei D15\AppData\Local\Temp\ipykernel_18108\2397153783.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  personality_data[col + "_avg"] = personality_data["tagged_words"].apply(
C:\Users\Huawei D15\AppData\Local\Temp\ipykernel_18108\2397153783.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  personality_data[col + "_avg"] = personality_data["tagged_words"].apply(
C:\Users\Huawei D15\AppData\Local\Temp\ipykernel_18108\2397153783.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

Stanford POS Stats Time: 91.40581035614014 seconds


C:\Users\Huawei D15\AppData\Local\Temp\ipykernel_18108\2397153783.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  personality_data[col + "_avg"] = personality_data["tagged_words"].apply(


In [28]:
# a quick look at the data
personality_data.head(2)

,type,is_Extrovert,is_Sensing,is_Thinking,is_Judging,posts,clean_posts,compound_sentiment,pos_sentiment,neg_sentiment,...,ADV_avg,CONJ_avg,DET_avg,NOUN_avg,NUM_avg,PRT_avg,PRON_avg,VERB_avg,._avg,X_avg
0,INFJ,0,0,0,1,'When asked of the things you wish you did ear...,when asked the things you wish you did earli...,0.99990,0.346041,0.128155,...,4.0,5.0,3.0,6.0,0.0,0.0,4.0,8.0,5.0,0.0
1,INFJ,0,0,0,1,'I love both and they are equally important to...,love both and they are equally important ...,0.99995,0.532258,0.143689,...,3.0,5.0,2.0,5.0,0.0,0.0,5.0,8.0,3.0,0.0


In [4]:
# Sentiment scoring & POS Tagging took long. So saving the scored & tagged file to save time in the next step.
personality_data.to_csv(os.path.join("..", "data", "clean_data_2.csv"), index=False)

<IPython.core.display.Javascript object>